In [ ]:
import pandas as pd
import mysql.connector

from datetime import datetime

conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Admin@123",
                               database = "dataspark")

cursor = conn.cursor()

In [ ]:
df_customer = pd.read_csv('Customers.csv', encoding='latin1')

df_customer.columns = df_customer.columns.str.lower()
df_customer.columns = df_customer.columns.str.replace(" ","_")

df_customer['birthday'] = pd.to_datetime(df_customer['birthday'])

def calculate_age(birthdate):
    today = datetime.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

df_customer['age'] = df_customer['birthday'].apply(calculate_age)

def age_segment(age):
    if age < 25:
        return 'young adult'
    elif age < 55:
        return 'middle age'
    else:
        return 'senior citizen'

# Apply the function to the 'age' column
df_customer['segment'] = df_customer['age'].apply(age_segment)

df_customer['state_code'] = df_customer['state_code'].fillna("NA")

In [ ]:
df_customer.head()

In [ ]:
insert_customer_values = '''
    INSERT INTO customer (
        customerkey,
        gender,
        name,
        city,
        state_code,
        state,
        zip_code,
        country,
        continent,
        birthday,
        age,
        segment
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        gender = VALUES(gender),
        name = VALUES(name),
        city = VALUES(city),
        state_code = VALUES(state_code),
        state = VALUES(state),
        zip_code = VALUES(zip_code),
        country = VALUES(country),
        continent = VALUES(continent),
        birthday = VALUES(birthday),
        age = VALUES(age),
        segment = VALUES(segment)
'''

# Insert data into the customer table
for index, row in df_customer.iterrows():
    cursor.execute(insert_customer_values, (
        row['customerkey'],
        row['gender'],
        row['name'],
        row['city'],
        row['state_code'],
        row['state'],
        row['zip_code'],
        row['country'],
        row['continent'],
        row['birthday'],
        row['age'],
        row['segment']
    ))

conn.commit()


In [ ]:
columns = ['customerkey', 'gender', 'name', 'city', 'state_code', 'state',
           'zip_code', 'country', 'continent', 'birthday', 'age', 'segment']

# Prefix to add
prefix = 'c.'

# Convert column names
converted_columns = [f'{prefix}{col}' for col in columns]
columns = ['c.'.join(column) for column in columns]

for i in converted_columns:
    print(i)